In [2]:
import pandas
import duckdb

In [4]:
childCareData = pandas.read_excel("nationaldatabaseofchildcareprices.xlsx")

In [5]:
subquery = duckdb.query("SELECT State_Name, County_Name, County_FIPS_Code, MIN(StudyYear) as minYear, MAX(StudyYear) as maxYear FROM childCareData WHERE MCInfant IS NOT NULL GROUP BY State_Name, County_Name, County_FIPS_Code").to_df()


In [19]:
query = """
SELECT SQ.*, 
  (CCD.MCinfant + CCD.MCToddler + CCD.MCPreschool + CCD.MCSA +
   CCD.MFCCInfant + CCD.MFCCToddler + CCD.MFCCPreschool + CCD.MFCCSA)/8.0 as startAvg
FROM subquery as SQ
LEFT JOIN childCareData as CCD
  ON SQ.State_Name = CCD.State_Name AND SQ.County_FIPS_Code = CCD.County_FIPS_Code AND SQ.minYear = CCD.StudyYear
"""
subquery = duckdb.query(query).to_df()

In [23]:
query = """
SELECT SQ.*, 
  (CCD.MCinfant + CCD.MCToddler + CCD.MCPreschool + CCD.MCSA +
   CCD.MFCCInfant + CCD.MFCCToddler + CCD.MFCCPreschool + CCD.MFCCSA)/8.0 as endAvg
FROM subquery as SQ
LEFT JOIN childCareData as CCD
  ON SQ.State_Name = CCD.State_Name AND SQ.County_FIPS_Code = CCD.County_FIPS_Code AND SQ.maxYear = CCD.StudyYear
"""
subquery = duckdb.query(query).to_df()

In [ ]:
result = duckdb.query("SELECT *, ((endAvg - startAvg)/startAvg) / (maxYear - minYear + 1) as avgYearIncrease FROM subquery").to_df()
result.to_csv("avgYearData.csv")